In [2]:
import sys; sys.path.append('../')
# load selectpercentile
from sklearn.feature_selection import SelectPercentile, f_classif
from cascade import CascadeClassifier
import numpy as np
import os
import pickle as pkl
import time
from feature_extractor import FeatureExtractor

In [3]:
parent = '../Features/'

In [4]:
%mkdir ../Features
feature_extractor = FeatureExtractor(shape=(19,19),
                                     percentile=10,
                                     all_features_file=parent + 'all_features.npz',
                                     selected_features_file=parent + 'selected_features.npz',
                                     labels_file=parent + 'labels.npy',
                                     indecies_file=parent + 'indecies.npy')

mkdir: cannot create directory ‘../Features’: File exists


In [ ]:
feature_extractor.extractFeatures(pos_path='../face.train/train/face', 
                                  neg_path='../face.train/train/non-face')
print('done')


all_features shape: (63960, 6977)
Now, saving all features to ../Features/all_features.npz
done


In [5]:
indecies, X = feature_extractor.selectPercentile()
y = np.load(parent + 'labels.npy')
print(X.shape, y.shape)

(6396, 6977) (6977,)


In [ ]:
# # do the following if the file selected_features is not present
# X, y, indecies = None, None, None
# if not os.path.exists('selected_features.npz'):
#     X = np.load('all_features.npz')['arr_0']
#     y = np.load('labels.npy')
#     selector = SelectPercentile(f_classif, percentile=10)
#     indecies = selector.fit(X.T, y).get_support(indices=True)
#     X = X[indecies, :]
#     np.savez('selected_features.npz', X)
#     # save indecies for later use
#     np.save('indecies.npy', indecies)
# else:
#     X = np.load('selected_features.npz')['arr_0']
#     y = np.load('labels.npy')
#     indecies = np.load('indecies.npy')
#     print(X.shape)
#     print(y.shape)

(6396, 6977)
(6977,)


In [ ]:
# !rm -r StrongClassifier
# !rm cascadeClassifier.pkl

In [18]:
%rm StrongClassifier/lastSC.last
s_t = time.time()
cascadeClassifier= CascadeClassifier(X, y, batchsize=1000, verbose=True, layers=[50])
print(np.sum(y == 1))
print(np.sum(y == 0))
acc = cascadeClassifier.train()
print('accuracy: ', acc)
print('time taken: ', time.time() - s_t, 's')
# save the classifier
cascadeClassifier.save(parent + 'cascadeClassifier.pkl')

2429
4548
$$$$$$$ Training layer 1 / 1 $$$$$$$
Starting to choose classifier
Finished training weak classifier 1 / 50
Starting to choose classifier
Finished training weak classifier 2 / 50
Starting to choose classifier
Finished training weak classifier 3 / 50
Starting to choose classifier
Finished training weak classifier 4 / 50
Starting to choose classifier
Finished training weak classifier 5 / 50
Starting to choose classifier
Finished training weak classifier 6 / 50
Starting to choose classifier
Finished training weak classifier 7 / 50
Starting to choose classifier
Finished training weak classifier 8 / 50
Starting to choose classifier
Finished training weak classifier 9 / 50
Starting to choose classifier
Finished training weak classifier 10 / 50
Starting to choose classifier
Finished training weak classifier 11 / 50
Starting to choose classifier
Finished training weak classifier 12 / 50
Starting to choose classifier
Finished training weak classifier 13 / 50
Starting to choose classif

In [19]:
s_t = time.time()
t_f_idx_map, t_features, t_labels = feature_extractor.extractFeaturesByIndecies(pos_path='../face.test/test/face/',
                                            neg_path='../face.test/test/non-face/',
                                            cascadeClassifier=cascadeClassifier)
print(t_features.shape, t_labels.shape)
predictions = cascadeClassifier.predict(t_features, t_f_idx_map)
print('test accuracy: ', np.sum(predictions == t_labels) / t_labels.shape[0])
print('time taken: ', time.time() - s_t, 's')

Now reading dataset...
Now extracting features from dataset...
(49, 23975) (23975,)
test accuracy:  0.9734723670490094
time taken:  1.0178771018981934 s


In [20]:
# report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

print('accuracy: ', accuracy_score(t_labels, predictions))
print('f1 score: ', f1_score(t_labels, predictions, average='macro'))
print('precision: ', precision_score(t_labels, predictions))
print('recall: ', recall_score(t_labels, predictions))
print('confusion matrix: \n', confusion_matrix(t_labels, predictions))
print('classification report: \n', classification_report(t_labels, predictions, digits=4))


accuracy:  0.9734723670490094
f1 score:  0.584518352898554
precision:  0.23202614379084968
recall:  0.1504237288135593
confusion matrix: 
 [[23268   235]
 [  401    71]]
classification report: 
               precision    recall  f1-score   support

           0     0.9831    0.9900    0.9865     23503
           1     0.2320    0.1504    0.1825       472

    accuracy                         0.9735     23975
   macro avg     0.6075    0.5702    0.5845     23975
weighted avg     0.9683    0.9735    0.9707     23975

